## C4.5 Algorithm (updated algorithm of ID3)

Here Instead of working on information gain, working with information gain RATIO . 
#### ADVANTAGE
1. Algorithms can have continuous values (humidity,Temp.)
2. The data csn have missing values in it.

In [6]:
import pandas as pd
data = [['Sunny',85,85,'Weak','No'],
        ['Sunny',80,90,'Strong','No'],
        ['Overcast',83,78,'Weak','Yes'],
        ['Rain',70,96,'Weak','Yes'],
        ['Rain',68,80,'Weak','Yes'],
        ['Rain',65,70,'Strong','No'],
        ['Overcast',64,65,'Strong','Yes'],
        ['Sunny',72,95,'Weak','No'],
        ['Sunny',69,70,'Weak','Yes'],
        ['Rain',75,80,'Weak','Yes'],
        ['Sunny',75,70,'Strong','Yes'],
        ['Overcast',72,90,'Strong','Yes'],
        ['Overcast',81,75,'Weak','Yes'],
        ['Rain',71,80,'Strong','No']]

In [7]:
title = ['Outlook','Temp.','Humidity','Wind','Decision']

In [8]:
Data = pd.DataFrame(data,columns = title)
Data

,Outlook,Temp.,Humidity,Wind,Decision
0,Sunny,85,85,Weak,No
1,Sunny,80,90,Strong,No
2,Overcast,83,78,Weak,Yes
3,Rain,70,96,Weak,Yes
4,Rain,68,80,Weak,Yes
5,Rain,65,70,Strong,No
6,Overcast,64,65,Strong,Yes
7,Sunny,72,95,Weak,No
8,Sunny,69,70,Weak,Yes
9,Rain,75,80,Weak,Yes


In [9]:
#creating uniques values for features
def unique_values(rows,col):
    return set([row[col] for row in rows])

In [13]:
unique_values(data,-1)

{'No', 'Yes'}

In [16]:
#counting the attributes of class label
def count_values(rows):
    count = {}
    for row in rows:
        label = row[-1]
        if label not in count:
            count[label] = 1
        else:
            count[label] += 1
    return count

In [17]:
count_values(data)

{'No': 5, 'Yes': 9}

In [21]:
#checkin whether the value is numeric or not
def is_numeric(val):
    return isinstance(val,int) or isinstance(val,float)


In [23]:
print(is_numeric('Red'))
print(is_numeric(7))

False
True


In [92]:
#creating Question
class Question:
    def __init__(self,col,value):
        self.col = col
        self.value = value
    
    def match(self,example):
        val = example[self.col]
        if is_numeric(self.value):
            return val <= self.value
        return val == self.value
        
        
    def __repr__(self):
        condition = '=='
        if is_numeric(self.value):
            condition = '<='
        return ('Is %s %s %s ?'%(title[self.col],condition,self.value))

In [93]:
q = Question(0,'Sunny')
q

Is Outlook == Sunny ?

In [94]:
r = Question(1,83)
r

Is Temp. <= 83 ?

In [201]:
# Splitting the dataset with respect to question
def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        #print(row)
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [96]:
t_r,f_r=partition(data,Question(3,'Weak'))

In [97]:
t_r

[['Sunny', 85, 85, 'Weak', 'No'],
 ['Overcast', 83, 78, 'Weak', 'Yes'],
 ['Rain', 70, 96, 'Weak', 'Yes'],
 ['Rain', 68, 80, 'Weak', 'Yes'],
 ['Sunny', 72, 95, 'Weak', 'No'],
 ['Sunny', 69, 70, 'Weak', 'Yes'],
 ['Rain', 75, 80, 'Weak', 'Yes'],
 ['Overcast', 81, 75, 'Weak', 'Yes']]

In [98]:
# Quantifyin methods

In Id3, We build the tree by calculating Entropy or gini and Information gain.    
In C4.5, We try to find the Information gain Ratio by providing splitting Information.

GainRatio(A) = Gain(A) / SplitInfo(A)   

SplitInfo(A) = -∑ |Dj|/|D| x log2|Dj|/|D|

In [99]:
#Impurity for splitting the data
def entropy(rows):
    count = count_values(rows)
    entropy = 0
    from math import log
    log2 = lambda x : log(x)/log(2)
    for label in count:
        p = count[label]/len(rows)
        entropy -= p * log2(p)
    return entropy

In [100]:
entropy(t_r)

0.8112781244591328

In [101]:
def info_gain_entropy(current,left,right):
    p = float(len(left))/(len(left)+len(right))
    return current - p*entropy(left) - (1-p)*entropy(right)

In [102]:
Impurity = entropy(data)
info_gain_entropy(Impurity,t_r,f_r)

0.04812703040826927

Going to find C4.5 algorithm important part..    
Splitting info ,Gain Ratio 

In [103]:
# Splitting Info
def split_Info(left,right):
    num = float(len(left))
    den = len(left)+len(right)
    p = num/den
    from math import log
    log2 = lambda x : log(x)/log(2)
    return -log2(p)*p - log2(1-p)*(1-p)

In [104]:
split_ratio(t_r,f_r)

0.9852281360342516

In [105]:
#Gain Ratio
def gainRatio(current_gain,split):
    return current_gain/split

In [106]:
gainRatio(info_gain_entropy(Impurity,t_r,f_r),split_ratio(t_r,f_r))

0.048848615511520595

In [164]:
#splitting the best question and gainRatio
def best_split(rows):
    best_gainRatio = 0
    best_question = None
    current = entropy(rows)
    features = len(rows[0])-1
    for col in range(features):
        value = unique_values(rows,col)
        for val in value:
            question = Question(col,val)
            
            true_rows,false_rows = partition(rows,question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            splitting_info = split_Info(true_rows,false_rows)
            
            gain = info_gain_entropy(current,true_rows,false_rows)
            GainRatio = gain/splitting_info
            if GainRatio > best_gainRatio:
                best_gainRatio,best_question = GainRatio,question
    return best_gainRatio,best_question

In [165]:
g,q = best_split(data)

In [166]:
q,g

(Is Temp. <= 83 ?, 0.30547141518417775)

In [159]:
#class --> decisiontree
class DecisionTree:
    def __init__(self,question,true_branch,false_branch):
        #stores question
        self.question = question
        #stores true_branch
        self.true_branch = true_branch
        #stores false_branch
        self.false_branch = false_branch

In [160]:
#class leaf
class Leaf:
    def __init__(self,rows):
        #stores the leaf value
        self.predictions = count_values(rows)

In [167]:
def build_tree(rows):
    #info_gain and question formed
    gainRatio,question = best_split(rows)
    
    #if gain = 0, then Leaf satisfied 
    if gainRatio == 0:
        return Leaf(rows)
    
    #to find a best value or question to partition on
    true_rows,false_rows = partition(rows,question)
    #recursive function to build the model
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return DecisionTree(question , true_branch , false_branch)


In [172]:
def print_tree(node,indentation=""):
    '''printing function'''
    #base case means we have reached the leaf
    #if the node object is of leaf type
    if isinstance(node,Leaf):
        print(indentation+"PREDICTION",node.predictions)
        return 
    #print the question at node
    print(indentation + str(node.question))
    
    #call the function on true branch 
    print(indentation+ "Return Yes")
    print_tree(node.true_branch,indentation + " ")
    
    #on flase branch
    print(indentation+ "Return No")
    print_tree(node.false_branch,indentation + " ")

In [173]:
my_tree = build_tree(data)

In [174]:
print_tree(my_tree)

Is Temp. <= 83 ?
Return Yes
 Is Outlook == Overcast ?
 Return Yes
  PREDICTION {'Yes': 4}
 Return No
  Is Temp. <= 65 ?
  Return Yes
   PREDICTION {'No': 1}
  Return No
   Is Temp. <= 75 ?
   Return Yes
    Is Temp. <= 70 ?
    Return Yes
     PREDICTION {'Yes': 3}
    Return No
     Is Temp. <= 72 ?
     Return Yes
      PREDICTION {'No': 2}
     Return No
      PREDICTION {'Yes': 2}
   Return No
    PREDICTION {'No': 1}
Return No
 PREDICTION {'No': 1}


In [197]:
testing_data = [['Rain',65,40,'Strong','Yes'],
        ['Sunny',75,70,'Strong','No'],
        ['Overcast',85,90,'Strong','Yes'],
        ['Rain',81,75,'Strong','No'],
        ['Rain',71,40,'Strong','No']]

In [198]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [199]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [200]:
print(title)
print()
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

['Outlook', 'Temp.', 'Humidity', 'Wind', 'Decision']

Actual: Yes. Predicted: {'No': '100%'}
Actual: No. Predicted: {'Yes': '100%'}
Actual: Yes. Predicted: {'No': '100%'}
Actual: No. Predicted: {'No': '100%'}
Actual: No. Predicted: {'No': '100%'}


### The Only difference is ,if there is a continuous values in the data , better using C4.5 Algorithm 
reduces the overfitting in the data
coz, it is calculated with redpect to the gainRatio(ci=ontinuous values)